In [1]:
import sys
if '..' not in sys.path:
    sys.path.append('..')


In [2]:
import os
import re
from typing import Any

import numpy as np
import pandas as pd
import reload
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision


In [3]:
%reload money_counter

from money_counter import data, engine, utils, coco_eval, models, constants
from money_counter.models import VersionManager
import money_counter.models
import money_counter.data
import money_counter


In [4]:
import logging

logging.basicConfig(level=logging.INFO)

In [5]:
MODEL_STATE_DIR = f'../model_state' # Relative to the notebook of the project
COINS_DATASET_PATH = os.environ['COINS_DATASET_PATH']


In [6]:
def add(df: pd.DataFrame, data) -> pd.DataFrame:
    last_idx = len(df)
    df.loc[last_idx] = data # type: ignore
    return df


In [7]:
def enumerate_states(model_state_dir, max=8):
    """
    Enumerates all the model states in the model_state_dir.
    :param model_state_dir: The directory where the model states are stored.
    :param max: The maximum number of model states to enumerate.
    :return: A generator of tuples (model name, epoch number, full path)
    """
    for model_name in os.listdir(model_state_dir):
        how_many = 0
        model_dir = os.path.join(model_state_dir, model_name)
        
        for model_state in reversed(os.listdir(model_dir)):
            path = os.path.join(model_dir, model_state)
            matches = re.match(".*epoch_(\\d+).pth", path)

            if matches:
                epoch_number = matches.group(1)

                yield model_name, int(epoch_number), path

                how_many += 1

                if how_many >= max:
                    break


In [8]:
import time

# Columns produced by the mAP metric
mAP_columns = [
    'map',
    'map_small',
    'map_medium',
    'map_large',
    'mar_1',
    'mar_10',
    'mar_100',
    'mar_small',
    'mar_medium',
    'mar_large',
    'map_50',
    'map_75',
    'map_per_class',
    'mar_100_per_class']

columns=['model', 'loss', 'epoch', 'path', *mAP_columns]

In [9]:
df = pd.DataFrame(columns=columns)

started_at = time.time()

for model_name, epoch_number, path in enumerate_states(MODEL_STATE_DIR):
    loaded = torch.load(path)
    # add to the dataframe
    row = {
        'model': model_name,
        'epoch': epoch_number,
        'loss': loaded['loss'],
        'path': path
    }

    df = add(df, row)

    if len(df) % 20 == 0:
        print(
            f'Processed {len(df)} states. Current speed: {len(df) / (time.time() - started_at)} states per second')


df = df.sort_values(by=['loss'])


Processed 20 states. Current speed: 1.3722946014623791 states per second


In [10]:
class ColorPerValue:
	"""Used to assign color to values"""
	def __init__(self):
		self._colors = ['red', 'orange', 'lightblue', 'lightgreen', 'white']
		self._values = []

	def get_color(self, value: Any):
		"""Returns the color for the value"""
		if value not in self._values:
			self._values.append(value)
		
		color_idx = self._values.index(value)		

		return self._colors[color_idx]

In [11]:
model_color = ColorPerValue()
# group by model and show the lowest losses
top_states = df.groupby('model').head(1).sort_values(by=['model', 'loss'])
# conditional coloring the first column based on name
top_states.style.apply(lambda x: [f'color: {model_color.get_color(x.model)}' for i in x], axis=1)


,model,loss,epoch,path,map,map_small,map_medium,map_large,mar_1,mar_10,mar_100,mar_small,mar_medium,mar_large,map_50,map_75,map_per_class,mar_100_per_class
3,fasterrcnn_resnet50_fpn-pretrained,0.296820,46,../model_state\fasterrcnn_resnet50_fpn-pretrained\epoch_046.pth,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
15,fasterrcnn_resnet50_fpn-untrained,0.353208,42,../model_state\fasterrcnn_resnet50_fpn-untrained\epoch_042.pth,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
20,fasterrcnn_resnet50_fpn_v2-pretrained,0.262157,45,../model_state\fasterrcnn_resnet50_fpn_v2-pretrained\epoch_045.pth,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
25,fasterrcnn_resnet50_fpn_v2-untrained,0.693418,48,../model_state\fasterrcnn_resnet50_fpn_v2-untrained\epoch_048.pth,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [12]:
from money_counter import data, engine, utils, coco_eval

version_manager = money_counter.models.VersionManager(MODEL_STATE_DIR)
device = utils.get_device()

_, data_loader_test = money_counter.data.get_data_loaders(COINS_DATASET_PATH, batch_size=4)

coco_evaluator = None


INFO:money_counter.data:Train dataset size: 487
INFO:money_counter.data:Test dataset size: 121


In [13]:
mAP = MeanAveragePrecision(num_classes=constants.NUM_CLASSES, iou_thresholds=[0.5, 0.75], max_detection_threshold=100, class_labels=constants.CLASSES)


In [14]:
from IPython import display

mAP.reset()

models_progress = display.ProgressBar(len(top_states))
models_progress.display()

images_progress = display.ProgressBar(len(data_loader_test))
images_progress.display()

# Let's calculate the mAP for the top states
for idx, row in top_states.iterrows():
    model_name = row['model']
    model = models.get_model(model_name).to(device)
    epoch_number = row['epoch']
    path = row['path']

    version_manager.load_model(
        model_name, model, epoch=row['epoch'], map_location=device)

    # Put the model in evaluation mode
    model.eval()

    with torch.no_grad():
        logging.info(f'Calculating mAP for {model_name} at epoch {epoch_number}...')

        images_progress.progress = 0
        images_progress.update()

        for (images, targets) in data_loader_test:
            # Move images to the device
            images = list(image.to(device) for image in images)
            # Move target to the device
            targets = [{k: v.to(device) for k, v in t.items()}
                       for t in targets]
            # update the progress 
            images_progress.progress += 1
            
            # Evaluate the model on the test set
            predictions = model(images)            

            # Update the mAP metric
            mAP.update(predictions, targets)

    # Update the mAP column in the row
    computed = mAP.compute()

    for key in computed:
        top_states.at[idx, key] = computed[key].item()

    models_progress.progress += 1
    models_progress.update()


[============================================================] 4/4

[================================================================================================================================================================================================================================================] 124/31

INFO:money_counter.models:Loaded model state from ../model_state/fasterrcnn_resnet50_fpn-pretrained/epoch_046.pth
INFO:root:Calculating mAP for fasterrcnn_resnet50_fpn-pretrained at epoch 46...
INFO:money_counter.models:Loaded model state from ../model_state/fasterrcnn_resnet50_fpn-untrained/epoch_042.pth
INFO:root:Calculating mAP for fasterrcnn_resnet50_fpn-untrained at epoch 42...
INFO:money_counter.models:Loaded model state from ../model_state/fasterrcnn_resnet50_fpn_v2-pretrained/epoch_045.pth
INFO:root:Calculating mAP for fasterrcnn_resnet50_fpn_v2-pretrained at epoch 45...
INFO:money_counter.models:Loaded model state from ../model_state/fasterrcnn_resnet50_fpn_v2-untrained/epoch_048.pth
INFO:root:Calculating mAP for fasterrcnn_resnet50_fpn_v2-untrained at epoch 48...


In [15]:
top_states.sort_values(by=['map'], ascending=False)

,model,loss,epoch,path,map,map_small,map_medium,map_large,mar_1,mar_10,mar_100,mar_small,mar_medium,mar_large,map_50,map_75,map_per_class,mar_100_per_class
3,fasterrcnn_resnet50_fpn-pretrained,0.296820,46,../model_state\fasterrcnn_resnet50_fpn-pretrai...,0.773670,-1.0,-1.0,0.773670,0.245821,0.822974,0.872282,-1.0,-1.0,0.872282,0.779582,0.767758,-1.0,-1.0
15,fasterrcnn_resnet50_fpn-untrained,0.353208,42,../model_state\fasterrcnn_resnet50_fpn-untrain...,0.766483,-1.0,-1.0,0.766490,0.246589,0.838961,0.896990,-1.0,-1.0,0.896990,0.773621,0.759344,-1.0,-1.0
20,fasterrcnn_resnet50_fpn_v2-pretrained,0.262157,45,../model_state\fasterrcnn_resnet50_fpn_v2-pret...,0.757615,-1.0,-1.0,0.757622,0.243761,0.836541,0.895520,-1.0,-1.0,0.895520,0.764216,0.751013,-1.0,-1.0
25,fasterrcnn_resnet50_fpn_v2-untrained,0.693418,48,../model_state\fasterrcnn_resnet50_fpn_v2-untr...,0.709994,-1.0,-1.0,0.710001,0.228269,0.799575,0.867934,-1.0,-1.0,0.867934,0.718844,0.701144,-1.0,-1.0
